In [ ]:
# Installs all the necessary packages for the project
%pip install pandas pandas-ta numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt
%pip install --upgrade certifi

In [2]:
# Imports all the necessary packages for the project and fixes ssl error
import ssl
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Get SP500 data    
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (pd.to_datetime(end_date) - pd.DateOffset(years=10)).strftime('%Y-%m-%d')

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()

df.index.names = ['Date', 'Symbol']

df

[*********************100%%**********************]  503 of 503 completed


Adj Close       Close        High         Low        Open  \
Date       Symbol                                                               
2014-01-27 A        38.307018   41.702431   41.752502   40.758224   41.416309   
           AAL      28.451939   30.180000   30.870001   28.670000   30.400000   
           AAPL     17.236380   19.660713   19.814285   19.491072   19.645357   
           ABBV     31.039011   46.830002   47.830002   46.419998   47.790001   
           ABT      29.952211   36.299999   36.830002   36.130001   36.680000   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

                        Volume  
Date       Symbol               
2014-01-27 A         6527262.0  
           AAL      16276300.0  
           AAPL    554878800.0  
           ABBV     10749800.0  
           ABT       9128900.0  
...                        ...  
2024-01-26 YUM       1157000.0  
           ZBH        982800.0  
           ZBRA       268300.0  
           ZION      1504900.0  
           ZTS       1840200.0  

[1236287 rows x 6 columns]

In [11]:
# Calculate features and technical indicators
# Garman-Klass Volatility, RSI, Bollinger Bands, ATR, MACD, Dollar Volume,
# All but RSI are normalized by subtracting the mean and dividing by the standard deviation

df['Garman-Klass'] = ((np.log(df['High'])-np.log(df['Low']))**2)/2-(2*np.log(2)-1)*((np.log(df['Adj Close'])-np.log(df['Open']))**2)  

df['RSI'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.rsi(close=x, length=20))

df['BB-Low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['BB-Mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['BB-High'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(data):
    atr = ta.atr(high=data['High'],
                        low=data['Low'],
                        close=data['Close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['ATR'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['MACD'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

df['Dollar Volume'] = (df['Adj Close']*df['Volume'])/1e6

df


Adj Close       Close        High         Low        Open  \
Date       Symbol                                                               
2014-01-27 A        38.307018   41.702431   41.752502   40.758224   41.416309   
           AAL      28.451939   30.180000   30.870001   28.670000   30.400000   
           AAPL     17.236380   19.660713   19.814285   19.491072   19.645357   
           ABBV     31.039011   46.830002   47.830002   46.419998   47.790001   
           ABT      29.952211   36.299999   36.830002   36.130001   36.680000   
...                       ...         ...         ...         ...         ...   
2024-01-26 YUM     129.089996  129.089996  130.690002  128.669998  129.919998   
           ZBH     121.690002  121.690002  123.110001  121.570000  122.839996   
           ZBRA    252.169998  252.169998  258.420013  251.619995  256.980011   
           ZION     44.020000   44.020000   44.860001   43.959999   44.500000   
           ZTS     191.520004  191.520004  192.880005  188.970001  189.210007   

                        Volume  Garman-Klass        RSI    BB-Low    BB-Mid  \
Date       Symbol                                                             
2014-01-27 A         6527262.0     -0.002062        NaN       NaN       NaN   
           AAL      16276300.0      0.001039        NaN       NaN       NaN   
           AAPL    554878800.0     -0.006476        NaN       NaN       NaN   
           ABBV     10749800.0     -0.071501        NaN       NaN       NaN   
           ABT       9128900.0     -0.015676        NaN       NaN       NaN   
...                        ...           ...        ...       ...       ...   
2024-01-26 YUM       1157000.0      0.000105  50.708962  4.856013  4.872770   
           ZBH        982800.0      0.000045  56.739863  4.794179  4.811249   
           ZBRA       268300.0      0.000218  51.191561  5.479828  5.545973   
           ZION      1504900.0      0.000160  57.948030  3.728311  3.786146   
           ZTS       1840200.0      0.000153  52.795286  5.230000  5.267848   

                    BB-High       ATR      MACD  Dollar Volume  
Date       Symbol                                               
2014-01-27 A            NaN       NaN       NaN     250.039945  
           AAL          NaN       NaN       NaN     463.092289  
           AAPL         NaN       NaN       NaN    9564.101642  
           ABBV         NaN       NaN       NaN     333.663160  
           ABT          NaN       NaN       NaN     273.430742  
...                     ...       ...       ...            ...  
2024-01-26 YUM     4.889527  0.169377  0.224806     149.357126  
           ZBH     4.828319 -0.523774  0.413233     119.596934  
           ZBRA    5.612118  0.287127 -0.006501      67.657211  
           ZION    3.843982  0.697197  0.630047      66.245699  
           ZTS     5.305696  0.931215 -0.322147     352.435112  

[1236287 rows x 14 columns]

In [14]:
# Aggregate to monthly data and filter top 150 most liquid stocks for each month

tech_cols = [c for c in df.columns.unique(0) if c not in ['Dollar Volume', 'Volume', 'Open',
                                                          'High', 'Low', 'Close']]

tech_data = (pd.concat([df.unstack('Symbol')['Dollar Volume'].resample('M').mean().stack('Symbol').to_frame('Dollar Volume'),
                   df.unstack()[tech_cols].resample('M').last().stack('Symbol')],
                  axis=1)).dropna()

tech_data

Dollar Volume   Adj Close  Garman-Klass        RSI  \
Date       Symbol                                                       
2014-03-31 A          126.759526   36.743198     -0.001417  46.245725   
           AAL        325.555864   34.504333     -0.000081  53.125752   
           AAPL      4002.947531   16.906151     -0.006513  52.864394   
           ABBV       200.336382   34.068008     -0.064865  52.876709   
           ABT        273.833567   31.775761     -0.014078  49.432499   
...                          ...         ...           ...        ...   
2024-01-31 YUM        196.809561  129.089996      0.000105  50.708962   
           ZBH        204.029004  121.690002      0.000045  56.739863   
           ZBRA        99.053216  252.169998      0.000218  51.191561   
           ZION       109.331953   44.020000      0.000160  57.948030   
           ZTS        363.109587  191.520004      0.000153  52.795286   

                     BB-Low    BB-Mid   BB-High       ATR      MACD  
Date       Symbol                                                    
2014-03-31 A       3.597938  3.639796  3.681655 -0.850551 -0.426921  
           AAL     3.534184  3.586000  3.637816  0.305313  0.250428  
           AAPL    2.862578  2.879370  2.896162 -1.037042 -0.206600  
           ABBV    3.538759  3.566867  3.594976 -1.155319 -0.069004  
           ABT     3.476131  3.505823  3.535514 -0.964832 -0.170534  
...                     ...       ...       ...       ...       ...  
2024-01-31 YUM     4.856013  4.872770  4.889527  0.169377  0.224806  
           ZBH     4.794179  4.811249  4.828319 -0.523774  0.413233  
           ZBRA    5.479828  5.545973  5.612118  0.287127 -0.006501  
           ZION    3.728311  3.786146  3.843982  0.697197  0.630047  
           ZTS     5.230000  5.267848  5.305696  0.931215 -0.322147  

[58430 rows x 9 columns]